In [11]:
# Install dependencies

%pip install -qU langchain langchain-openai langchain-community pypdf

Note: you may need to restart the kernel to use updated packages.


Set the environment variables. We need an API key and an LLM server, at minimum.

In [2]:
import os

# We will add the api key as an environment variable. 
# You can set this value directrly form the shell,
# instead of explicitly setting it on code.
os.environ["OPENAI_API_KEY"] = "..."
# We need a custom endpoint, as we will be calling Verde's LLM
API_ENDPOINT = "https://llm1.cyverse.ai/v1"

The first step is to create a `Chat` object. This will allow us to call it through langchain and later on using `LCEL`

In [3]:
from langchain_openai import ChatOpenAI 		# We use the OpenAI protocol, but are using another provider (Verde)

# We will connect to Mistral Instruct v0.3 through Verde
# Notice how we need to specify the API endpoint
model = ChatOpenAI(model="Mistral-7B-Instruct-v0.3", base_url=API_ENDPOINT)

# Do a test call
from pprint import pprint
response = model.invoke("Hello, who are you?")

pprint(response)

AIMessage(content=" Hello! I am a model of an artificial intelligence designed to assist with a variety of tasks. I don't have personal experiences or emotions, but I'm here to help you with your inquiries and questions to the best of my ability. How can I assist you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 9, 'total_tokens': 68}, 'model_name': 'mistralai/Mistral-7B-Instruct-v0.3', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-86980541-8e4b-4218-8271-8c58f1beece9-0', usage_metadata={'input_tokens': 9, 'output_tokens': 59, 'total_tokens': 68})


Chat applications build upon the message history paradigm. There are three types of messages:

- System
- Human
- AI

The message history represents the conversation between the LLM and the user. The `System` message is optional. When present, it is usually used to define the _persona_ of the LLM. For example, establishing a role, defining a task, etc.

Then, the history is composed of alternating `Human` - `AI` messages.

In [4]:
# Let's build a conversation history

from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a software developer, expert in the use of LLMs and langchain. Help the user answering the questions using didactic examples"),										# Use the system to establish the task of the LLM
    HumanMessage(content="What is Mistral and why would I need to use it"),		# This is the first "Human" message
]

# Instead of invoking with a string, pass the whole message history
response = model.invoke(messages)

# Add the response to the history
messages.append(response)

# Let's peek into the response
print(response.content)


 Mistral is an open-source language model toolkit developed by Meta (formerly Facebook). It's designed for researchers and engineers working on large-scale language models. Mistral provides a flexible and scalable framework for training, inference, and evaluation of transformer-based language models.

You might need to use Mistral if you're involved in the following tasks:

1. **Language Model Training**: If you're working on developing or improving a language model, Mistral allows you to experiment with different architectures, learning rates, and other parameters to find the best model for your specific use case.

2. **Multi-GPU Training**: Mistral supports distributed training across multiple GPUs, which can significantly speed up the training process for large models.

3. **Data Parallelism**: Mistral uses data parallelism to improve the training process by splitting the dataset across multiple GPUs. This can help to reduce the training time and memory requirements.

4. **Efficient

Observe how the return value of the LLM model is an `AIMessage` instance. We add that response to the message history, which will be used for a follow up question

In [5]:
pprint(messages)

[SystemMessage(content='You are a software developer, expert in the use of LLMs and langchain. Help the user answering the questions using didactic examples'),
 HumanMessage(content='What is Mistral and why would I need to use it'),
 AIMessage(content=" Mistral is an open-source language model toolkit developed by Meta (formerly Facebook). It's designed for researchers and engineers working on large-scale language models. Mistral provides a flexible and scalable framework for training, inference, and evaluation of transformer-based language models.\n\nYou might need to use Mistral if you're involved in the following tasks:\n\n1. **Language Model Training**: If you're working on developing or improving a language model, Mistral allows you to experiment with different architectures, learning rates, and other parameters to find the best model for your specific use case.\n\n2. **Multi-GPU Training**: Mistral supports distributed training across multiple GPUs, which can significantly speed 

Now, let's use a follow up query that relies on the coneversation history.

In [6]:
messages.append(HumanMessage("Give me a short example of a python script that calls the aforementioned LLM"))

response = model.invoke(messages)
messages.append(response)

print(response.content)

 To provide a short example of using a language model (LLM) with Mistral in Python, let's create a script that generates a greeting for a given name. We will use the T5 model, which is supported by Mistral, to perform this task.

First, make sure you have the Mistral and transformers packages installed:

```bash
pip install mistral transformers
```

Next, create a Python script called `greeting.py` and paste the following code:

```python
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from mistral.experiment import Experiment
from mistral.executor import SimpleExecutor
from mistral.opt import AdamW

def main():
    # Load the T5 model and tokenizer
    model_name = "t5-small"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # Define the model input and output
    input_text = f"Hello, my name is {"John"}. How can I assist you today?"
    target_text = "What's up, John!"

    # Tokeni

Notice how we didn't explicity refered to Mistral, instead it was picked up from the message history.

Writting messages can get really tedious. We make use of message templates to simplify tasks by parameterizing the values.

We will use a template to create a document summarizer built on top of the LLM

In [7]:
# Import the chat template
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_messages(
	[("system", "Summarize the document provided by the user into a succint, at most three-sentence statement"),
	("user", "{text}")]
)

document = """
The Congress shall have Power To lay and collect Taxes, Duties, Imposts and Excises, to pay the Debts and provide for the common Defence and general Welfare of the United States; but all Duties, Imposts and Excises shall be uniform throughout the United States;
To borrow Money on the credit of the United States;
To regulate Commerce with foreign Nations, and among the several States, and with the Indian Tribes;
To establish an uniform Rule of Naturalization, and uniform Laws on the subject of Bankruptcies throughout the United States;
To coin Money, regulate the Value thereof, and of foreign Coin, and fix the Standard of Weights and Measures;
To provide for the Punishment of counterfeiting the Securities and current Coin of the United States;
To establish Post Offices and post Roads;
To promote the Progress of Science and useful Arts, by securing for limited Times to Authors and Inventors the exclusive Right to their respective Writings and Discoveries;
To constitute Tribunals inferior to the supreme Court;
To define and punish Piracies and Felonies committed on the high Seas, and Offences against the Law of Nations;
To declare War, grant Letters of Marque and Reprisal, and make Rules concerning Captures on Land and Water;
To raise and support Armies, but no Appropriation of Money to that Use shall be for a longer Term than two Years;
To provide and maintain a Navy;
To make Rules for the Government and Regulation of the land and naval Forces;
To provide for calling forth the Militia to execute the Laws of the Union, suppress Insurrections and repel Invasions;
To provide for organizing, arming, and disciplining, the Militia, and for governing such Part of them as may be employed in the Service of the United States, reserving to the States respectively, the Appointment of the Officers, and the Authority of training the Militia according to the discipline prescribed by Congress;
To exercise exclusive Legislation in all Cases whatsoever, over such District (not exceeding ten Miles square) as may, by Cession of particular States, and the Acceptance of Congress, become the Seat of the Government of the United States, and to exercise like Authority over all Places purchased by the Consent of the Legislature of the State in which the Same shall be, for the Erection of Forts, Magazines, Arsenals, dock-Yards, and other needful Buildings;—And
To make all Laws which shall be necessary and proper for carrying into Execution the foregoing Powers, and all other Powers vested by this Constitution in the Government of the United States, or in any Department or Officer thereof.
"""

prompt = prompt_template.invoke({"text": document})
response = model.invoke(prompt)

pprint(prompt)
print()
pprint(response.content)

ChatPromptValue(messages=[SystemMessage(content='Summarize the document provided by the user into a succint, at most three-sentence statement'), HumanMessage(content='\nThe Congress shall have Power To lay and collect Taxes, Duties, Imposts and Excises, to pay the Debts and provide for the common Defence and general Welfare of the United States; but all Duties, Imposts and Excises shall be uniform throughout the United States;\nTo borrow Money on the credit of the United States;\nTo regulate Commerce with foreign Nations, and among the several States, and with the Indian Tribes;\nTo establish an uniform Rule of Naturalization, and uniform Laws on the subject of Bankruptcies throughout the United States;\nTo coin Money, regulate the Value thereof, and of foreign Coin, and fix the Standard of Weights and Measures;\nTo provide for the Punishment of counterfeiting the Securities and current Coin of the United States;\nTo establish Post Offices and post Roads;\nTo promote the Progress of Sc

We had to call `invoke` times: one for the prompt template and the other on the model. We can streamline this by _chaining_ them together

In [8]:
chain = prompt_template | model

response = chain.invoke({"text": document})

pprint(response.content)

(' The U.S. Congress has the power to collect taxes, regulate commerce, '
 'establish a uniform rule of naturalization, coin money, establish post '
 'offices, promote science and the arts, declare war, raise armies and navies, '
 'govern militia, and make all necessary laws for executing these powers as '
 'outlined in the U.S. Constitution. Additionally, the Congress holds '
 'exclusive legislation over the District of Columbia and places purchased for '
 'federal buildings.')


We still get as a response an `AIMessage` object. We can use _output parsers_ to transform messages into a specific format. The most basic is `StrOutputParser`.

There are other parsers to handle Json, Http responses, etc.

In [9]:
from langchain_core.output_parsers import StrOutputParser

str_parser = StrOutputParser()

chain = prompt_template | model | str_parser

response = chain.invoke({"text": document})

# Notice how we don't have to call "content" any more
pprint(response)

(' The U.S. Constitution grants Congress the power to:\n'
 '1. Impose and collect taxes, borrow money, and regulate commerce '
 'domestically and internationally.\n'
 '2. Establish the U.S. monetary system, coin money, and regulate foreign '
 'currency.\n'
 '3. Establish post offices, promote science and useful arts, create courts, '
 'and make laws necessary for the execution of its powers. Additionally, it '
 'grants specific powers regarding defense, naturalization, bankruptcies, '
 'piracy, war, militia, and exclusive legislation over federal territories.')


The `|` operator that connects together the individual comonents is part of `LCEL`. Langchain's declarative syntax to define chains: pipelines of components that orchestrate data flow.

Let's build a sample _chain_ to generate a json object that represents an invoice from a pdf file.

In [25]:
from langchain_community.document_loaders import PyPDFLoader

# Sample PDF invoice
pdf_path = "sample-invoice.pdf"


def read_pdf(file_path:str) -> str:
	loader = PyPDFLoader(file_path)
	pages = loader.load()

	return {"text": '\n'.join([p.page_content for p in pages])}

pprint(read_pdf(pdf_path))

{'text': 'CPB Software (Germany) GmbH - Im Bruch 3 - 63897 Miltenberg/Main\n'
         'Musterkunde AG\n'
         'Mr. John Doe\n'
         'Musterstr. 23\n'
         '12345 Musterstadt Name:  Stefanie Müller\n'
         'Phone: +49 9371 9786-0\n'
         'Invoice WMACCESS Internet\n'
         'VAT No. DE199378386\n'
         'Invoice No\n'
         '123100401\n'
         'Amount\n'
         '-without VAT-quantity\n'
         '130,00 € 1\n'
         '10,00 € 0\n'
         '50,00 € 0\n'
         '1.000,00 € 0\n'
         '10,00 € 0\n'
         '0,58 € 14\n'
         '0,70 € 0\n'
         '1,50 € 162\n'
         '0,50 € 0\n'
         '0,80 € 0\n'
         '1,80 € 0\n'
         '0,30 € 0\n'
         '0,30 € 0\n'
         '0,40 € 0\n'
         '0,40 € 0\n'
         '0,30 € 0\n'
         '0,30 € 0\n'
         'Terms of Payment: Immediate payment without discount. Any bank '
         'charges must be paid by the invoice recipient.\n'
         'Bank fees at our expense will be charged to th

In [26]:
# Make our function a langchain component to be compatitle with LCEL
from langchain_core.runnables import RunnableLambda

pdf_reader = RunnableLambda(read_pdf)

pprint(pdf_reader.invoke(pdf_path))

{'text': 'CPB Software (Germany) GmbH - Im Bruch 3 - 63897 Miltenberg/Main\n'
         'Musterkunde AG\n'
         'Mr. John Doe\n'
         'Musterstr. 23\n'
         '12345 Musterstadt Name:  Stefanie Müller\n'
         'Phone: +49 9371 9786-0\n'
         'Invoice WMACCESS Internet\n'
         'VAT No. DE199378386\n'
         'Invoice No\n'
         '123100401\n'
         'Amount\n'
         '-without VAT-quantity\n'
         '130,00 € 1\n'
         '10,00 € 0\n'
         '50,00 € 0\n'
         '1.000,00 € 0\n'
         '10,00 € 0\n'
         '0,58 € 14\n'
         '0,70 € 0\n'
         '1,50 € 162\n'
         '0,50 € 0\n'
         '0,80 € 0\n'
         '1,80 € 0\n'
         '0,30 € 0\n'
         '0,30 € 0\n'
         '0,40 € 0\n'
         '0,40 € 0\n'
         '0,30 € 0\n'
         '0,30 € 0\n'
         'Terms of Payment: Immediate payment without discount. Any bank '
         'charges must be paid by the invoice recipient.\n'
         'Bank fees at our expense will be charged to th

In [28]:
# Create a chain to process the contents of the PDF

prompt_template = ChatPromptTemplate.from_messages(
	[("system", "You are going to read the contents of an invoice in PDF format. Return a JSON object that contains the data of the invoice using the fields and values from the text below"),
	 ("user", "{text}")]
)

# We will look at 
from langchain_core.output_parsers import JsonOutputParser

chain = pdf_reader | prompt_template | model | JsonOutputParser()

invoice = chain.invoke(pdf_path)

pprint(invoice)

{'additional_information': 'The explanation of the query fee categories (T1 to '
                           'T6 and G1 to G6) can be found on our website: '
                           'https://www.wmaccess.com/abfragekategorien',
 'biller': {'address': {'city': 'Miltenberg/Main',
                        'country': 'Germany',
                        'postal_code': '63897',
                        'street': 'Im Bruch 3'},
            'name': 'CPB Software (Germany) GmbH'},
 'charges': [{'amount': 0, 'type': 'Transaction Fee T6'},
             {'amount': 0, 'type': 'Basic Fee wmPos'},
             {'charges': [{'amount': 154.3, 'user_account': 'user-account-1'},
                          {'amount': 96.82, 'user_account': 'user-account-2'}],
              'type': 'Change of user accounts'},
             {'amount': 0, 'type': 'Transaction Fee T101.02.2024 - 29.02.2024'},
             {'amount': 0, 'type': 'Transaction Fee G6'},
             {'amount': 0, 'type': 'Transaction Fee G3'},
     

In [29]:
# See how JSON parser create a python dictionary out of JSON data produced by the LLM
type(invoice)

dict